In [1]:
# ==============================================
#  UNIFICAR TABLAS DEL MODELO EPISCOPE
# ==============================================

import pandas as pd
import episcopeenvigado.dataset as ds

# ------------------------------------------------
# Función: unificar_dataset
# Une fact_atenciones con las tablas de dimensiones
# sin eliminar ninguna columna del fact
# ------------------------------------------------
def unificar_dataset(episcope_data):
    # Extraer tablas individuales del diccionario
    fact = episcope_data["fact_atenciones"]
    dim_causa = episcope_data["dim_causa_ext"]
    dim_cie10 = episcope_data["dim_cie10"]
    dim_depto = episcope_data["dim_departamento"]
    dim_estado = episcope_data["dim_estado_salida"]
    dim_mpio = episcope_data["dim_municipio"]
    dim_via = episcope_data["dim_via_ingreso"]

    # Copiar fact para no modificar el original
    df = fact.copy()

    # --- Unir dimensiones al fact ---
    df = df.merge(dim_causa, on="causa_ext_id", how="left")
    df = df.merge(dim_depto, on="departamento_id", how="left")
    df = df.merge(dim_mpio, on="municipio_id", how="left")
    df = df.merge(dim_estado, on="estado_salida_id", how="left")
    df = df.merge(dim_via, on="via_ingreso_id", how="left")

    # --- Diagnóstico principal (CIE10) ---
    df = df.merge(
        dim_cie10[["cie_4cat", "desc_4cat", "nombre_cap"]],
        left_on="Cod_Dx_Ppal_Egreso",
        right_on="cie_4cat",
        how="left"
    )

    # --- Renombrar columnas descriptivas ---
    df = df.rename(columns={
        "desc_4cat": "Diagnostico_Principal_Desc",
        "nombre_cap": "Capitulo_CIE10",
        "causa_ext_desc": "Causa_Externa_Desc",
        "departamento_desc": "Departamento_Desc",
        "municipio_desc": "Municipio_Desc",
        "estado_salida_desc": "Estado_Salida_Desc",
        "via_ingreso_desc": "Via_Ingreso_Desc",
    })

    return df


# ==============================================
#  USO DE LA FUNCIÓN
# ==============================================

# 1. Cargar los datasets desde el módulo interno
episcope_data = ds.obtener_dataset_completo()

# 2. Unificar todas las tablas
df_unificado = unificar_dataset(episcope_data)

# 3. Revisar resultado
print(f"Dataset unificado: {df_unificado.shape[0]} filas, {df_unificado.shape[1]} columnas")
df_unificado.head()


2025-11-05 13:21:52.712 | INFO     | episcopeenvigado.config:<module>:11 - PROJ_ROOT path is: C:\Users\ZENBOOK\OneDrive - MUNICIPIO DE ENVIGADO\Documentos\Personales\Analisis de datos avanzado\EpiScopeEnvigado_Final\EpiScopeEnvigado


2025-11-05 13:21:53.293 | INFO     | episcopeenvigado.dataset:obtener_dataset_completo:110 - 📋 Se encontraron 7 tablas en la base de datos.
2025-11-05 13:21:53.305 | SUCCESS  | episcopeenvigado.dataset:obtener_dataset_completo:119 - ✅ Tabla 'dim_causa_ext' cargada correctamente (14 filas).
2025-11-05 13:21:53.631 | SUCCESS  | episcopeenvigado.dataset:obtener_dataset_completo:119 - ✅ Tabla 'dim_cie10' cargada correctamente (12568 filas).
2025-11-05 13:21:53.636 | SUCCESS  | episcopeenvigado.dataset:obtener_dataset_completo:119 - ✅ Tabla 'dim_departamento' cargada correctamente (34 filas).
2025-11-05 13:21:53.640 | SUCCESS  | episcopeenvigado.dataset:obtener_dataset_completo:119 - ✅ Tabla 'dim_estado_salida' cargada correctamente (2 filas).
2025-11-05 13:21:53.656 | SUCCESS  | episcopeenvigado.dataset:obtener_dataset_completo:119 - ✅ Tabla 'dim_municipio' cargada correctamente (1124 filas).
2025-11-05 13:21:53.660 | SUCCESS  | episcopeenvigado.dataset:obtener_dataset_completo:119 - ✅ Tab

,fact_id,Cod_IPS,ID,Fecha_Ingreso,Fecha_Egreso,Duracion_Dias,via_ingreso_id,estado_salida_id,municipio_id,causa_ext_id,...,municipio_dane,departamento_cod_y,Municipio_Desc,estado_salida_cod,Estado_Salida_Desc,via_ingreso_cod,Via_Ingreso_Desc,cie_4cat,Diagnostico_Principal_Desc,Capitulo_CIE10
0,1,52660212605,PAC00001,2023-09-04,2023-09-14,10,3,1,114,12,...,05266,05,ENVIGADO,1,1,3,REMITIDO,F28X,Otros trastornos psicoticos de origen no orga...,Trastornos mentales y del comportamiento (F00-...
1,2,52660212605,PAC00002,2023-02-05,2023-02-28,23,2,1,114,12,...,05266,05,ENVIGADO,1,1,2,CONSULTA EXTERNA,A971,Dengue con signos de alarma,Ciertas enfermedades infecciosas y parasitaria...
2,3,52660212605,PAC00003,2023-09-30,2023-09-30,0,3,1,1,12,...,05001,05,MEDELLÍN,1,1,3,REMITIDO,B159,"Hepatitis aguda tipo A, sin coma hepatico",Ciertas enfermedades infecciosas y parasitaria...
3,4,52660212605,PAC00004,2023-08-31,2023-09-29,29,3,1,1,12,...,05001,05,MEDELLÍN,1,1,3,REMITIDO,G042,Meningoencefalitis y meningomielitis bacterian...,Enfermedades del sistema nervioso (G00-G99)
4,5,52660212605,PAC00005,2023-08-26,2023-09-01,6,3,1,1,2,...,05001,05,MEDELLÍN,1,1,3,REMITIDO,S822,Fractura de la diafisis de la tibia,"Traumatismos, envenenamiento y algunas otras c..."


In [2]:
# Revisar estructura general
df_unificado.info()

# Verificar si hay datos faltantes en columnas claves
df_unificado[['Diagnostico_Principal_Desc', 'Municipio_Desc', 'Via_Ingreso_Desc']].isna().mean()

# Vista previa de las primeras filas
df_unificado.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45303 entries, 0 to 45302
Data columns (total 42 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   fact_id                     45303 non-null  int64  
 1   Cod_IPS                     45303 non-null  object 
 2   ID                          45303 non-null  object 
 3   Fecha_Ingreso               45303 non-null  object 
 4   Fecha_Egreso                45303 non-null  object 
 5   Duracion_Dias               45303 non-null  int64  
 6   via_ingreso_id              45303 non-null  int64  
 7   estado_salida_id            45303 non-null  int64  
 8   municipio_id                45303 non-null  int64  
 9   causa_ext_id                45303 non-null  int64  
 10  departamento_id             45303 non-null  int64  
 11  VIA INGRESO                 45303 non-null  int64  
 12  Estado_Salida               45303 non-null  object 
 13  EDAD                        453

,fact_id,Cod_IPS,ID,Fecha_Ingreso,Fecha_Egreso,Duracion_Dias,via_ingreso_id,estado_salida_id,municipio_id,causa_ext_id,...,municipio_dane,departamento_cod_y,Municipio_Desc,estado_salida_cod,Estado_Salida_Desc,via_ingreso_cod,Via_Ingreso_Desc,cie_4cat,Diagnostico_Principal_Desc,Capitulo_CIE10
0,1,52660212605,PAC00001,2023-09-04,2023-09-14,10,3,1,114,12,...,05266,05,ENVIGADO,1,1,3,REMITIDO,F28X,Otros trastornos psicoticos de origen no orga...,Trastornos mentales y del comportamiento (F00-...
1,2,52660212605,PAC00002,2023-02-05,2023-02-28,23,2,1,114,12,...,05266,05,ENVIGADO,1,1,2,CONSULTA EXTERNA,A971,Dengue con signos de alarma,Ciertas enfermedades infecciosas y parasitaria...
2,3,52660212605,PAC00003,2023-09-30,2023-09-30,0,3,1,1,12,...,05001,05,MEDELLÍN,1,1,3,REMITIDO,B159,"Hepatitis aguda tipo A, sin coma hepatico",Ciertas enfermedades infecciosas y parasitaria...
3,4,52660212605,PAC00004,2023-08-31,2023-09-29,29,3,1,1,12,...,05001,05,MEDELLÍN,1,1,3,REMITIDO,G042,Meningoencefalitis y meningomielitis bacterian...,Enfermedades del sistema nervioso (G00-G99)
4,5,52660212605,PAC00005,2023-08-26,2023-09-01,6,3,1,1,2,...,05001,05,MEDELLÍN,1,1,3,REMITIDO,S822,Fractura de la diafisis de la tibia,"Traumatismos, envenenamiento y algunas otras c..."
5,6,52660212605,PAC00006,2023-10-16,2023-10-20,4,3,1,95,12,...,05129,05,CALDAS,1,1,3,REMITIDO,N390,"Infeccion de vias urinarias, sitio no especifi...",Enfermedades del sistema genitourinario (N00-N99)
6,7,52660212605,PAC00007,2023-10-22,2023-10-29,7,3,1,75,12,...,05088,05,BELLO,1,1,3,REMITIDO,F111,Trastornos mentales y del comportamiento debid...,Trastornos mentales y del comportamiento (F00-...
7,8,52660212605,PAC00008,2023-08-31,2023-09-03,3,3,1,99,12,...,05360,05,ITAGÜÍ,1,1,3,REMITIDO,M545,Lumbago no especificado,Enfermedades del sistema osteomuscular y del t...
8,9,52660212605,PAC00009,2023-06-29,2023-07-02,3,3,1,114,12,...,05266,05,ENVIGADO,1,1,3,REMITIDO,N390,"Infeccion de vias urinarias, sitio no especifi...",Enfermedades del sistema genitourinario (N00-N99)
9,10,52660212605,PAC00010,2023-07-31,2023-08-05,5,3,1,114,12,...,05266,05,ENVIGADO,1,1,3,REMITIDO,Q282,Malformacion arteriovenosa de los vasos cerebr...,"Malformaciones congenitas, deformidades y anom..."


In [5]:
# ==========================================
#  Gráfico de torta: Distribución por vía de ingreso
# ==========================================

import pandas as pd
import plotly.express as px
import plotly.io as pio
import sys, subprocess

# 1️⃣ Verificar que nbformat esté instalado
try:
    import nbformat
except ImportError:
    print("⚙️ Instalando nbformat en el entorno virtual...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "nbformat"])
    import nbformat

# 2️⃣ Configurar renderer para abrir en navegador
pio.renderers.default = 'browser'

# 3️⃣ Calcular frecuencia de la columna correcta
frecuencia_via = (
    df_unificado["Via_Ingreso_Desc"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "Via_Ingreso_Desc", "Via_Ingreso_Desc": "Frecuencia"})
)

# 4️⃣ Crear gráfico de torta
fig = px.pie(
    frecuencia_via,
    names='Via_Ingreso_Desc',
    values='Frecuencia',
    title='Distribución de las vías de ingreso',
    color_discrete_sequence=px.colors.qualitative.Set2
)

# Mostrar porcentaje y valor absoluto
fig.update_traces(textinfo='label+percent+value')

# 5️⃣ Mostrar gráfico
fig.show()


ValueError: Value of 'names' is not the name of a column in 'data_frame'. Expected one of ['Frecuencia', 'count'] but received: Via_Ingreso_Desc